In [1]:
import os
import torch
from moviepy import VideoFileClip
from transformers import pipeline, AutoTokenizer
import whisper
from ultralytics import YOLO
import cv2
from multiprocessing import Pool

d:\src_git\curiosity\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Ensure CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [3]:
# Extract audio and frames
def extract_audio_and_frames(video_path, frame_rate=1):
    clip = VideoFileClip(video_path)
    audio_path = "audio.wav"
    clip.audio.write_audiofile(audio_path)
    frames = []
    for t in range(0, int(clip.duration), frame_rate):
        frame = clip.get_frame(t)
        frames.append(frame)
    return audio_path, frames

In [4]:
# Transcribe audio using Whisper
def transcribe_audio(audio_path):
    model = whisper.load_model("base").to(device)
    result = model.transcribe(audio_path)
    return result['text']

In [5]:
from transformers import AutoTokenizer, pipeline

def detect_explicit_language(text):
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    # classifier = pipeline("text-classification", model="bhadresh-savani/distilbert-base-uncased-emotion", device=0)
    classifier = pipeline("text-classification", model="allenai/longformer-base-4096", device=0)

    
    # max_length = 512
    max_length = 1024  # Literal experiment.
    tokens = tokenizer(text, return_tensors="pt", truncation=False)['input_ids']
    
    # Split tokens into chunks of 512
    chunks = torch.split(tokens, max_length, dim=1)
    
    explicit_results = []
    for chunk in chunks:
        truncated_text = tokenizer.decode(chunk.squeeze().tolist(), skip_special_tokens=True)
        results = classifier(truncated_text)
        explicit_results.extend([res for res in results if res['label'] == 'EXPLICIT'])
    return explicit_results


In [6]:
# Preprocess and detect explicit visuals in frames
def preprocess_frame(frame, target_size=(640, 640)):
    return cv2.resize(frame, target_size)

In [7]:
def detect_explicit_frames(frames):
    model = YOLO("yolov8n.pt")  # Replace with fine-tuned model
    explicit_frames = []
    for idx, frame in enumerate(frames):
        frame = preprocess_frame(frame)
        results = model.predict(frame, device=device)
        if any(res.name == "explicit" for res in results[0].boxes):
            explicit_frames.append(idx)
    return explicit_frames

In [8]:
# Analyze video and combine results
def analyze_video(video_path):
    audio_path, frames = extract_audio_and_frames(video_path)
    
    # Audio analysis
    transcription = transcribe_audio(audio_path)
    explicit_text = detect_explicit_language(transcription)
    
    # Visual analysis
    explicit_frames = detect_explicit_frames(frames)
    
    return {"explicit_text": explicit_text, "explicit_frames": explicit_frames}

In [9]:
# Save results
def save_results(results, output_path="explicit_timings.txt"):
    with open(output_path, "w") as f:
        for key, value in results.items():
            f.write(f"{key}: {value}\n")

In [10]:
# Main function
if __name__ == "__main__":
    video_path = "E:\\Movies\\GWLG\\GWLG.mkv"  # Replace with your video file
    results = analyze_video(video_path)
    save_results(results)
    print("Analysis complete. Results saved to explicit_timings.txt.")

MoviePy - Writing audio in audio.wav


MoviePy - Done.


d:\src_git\curiosity\venv\Lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kumar\.cache\huggingface\hub\models--allenai--longformer-base-4096. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/long

100%|██████████| 6.25M/6.25M [00:00<00:00, 67.8MB/s]



0: 640x640 (no detections), 5.5ms
Speed: 2.0ms preprocess, 5.5ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 6.5ms
Speed: 1.0ms preprocess, 6.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 5.0ms
Speed: 1.0ms preprocess, 5.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 5.0ms
Speed: 1.0ms preprocess, 5.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 4.0ms
Speed: 1.5ms preprocess, 4.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 6.0ms
Speed: 1.0ms preprocess, 6.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 9.0ms
Speed: 2.0ms preprocess, 9.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 6.0ms
Speed: 1.5ms preprocess, 6.0ms inference, 0.5ms 

NotImplementedError: Could not run 'torchvision::nms' with arguments from the 'CUDA' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'torchvision::nms' is only available for these backends: [CPU, Meta, QuantizedCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMeta, Tracer, AutocastCPU, AutocastXPU, AutocastMPS, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

CPU: registered at C:\actions-runner\_work\vision\vision\pyt